In [1]:
%run types.ipynb

%run gcal.ipynb

In [ ]:
# imports
from datetime import date, datetime, timedelta

import ipywidgets as widgets
from multiprocessing import Process
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# set of calendar activities shared across visualizations
activities = {}

In [3]:
# configuration parameters

CALENDARS = ["Focused Work", "Meeting"]  # set of calendars to visualize
ACTIVITY_REGEX = r"(?P<category>.+?(?= - |$))( - (?P<activity>.+))?"  # regex to capture category and activity from event names
CACHE_FILE = "/tmp/token.pickle"  # local path to where token should be stored
CREDENTIALS_FILE = os.getenv(
    "CALSTATS_CREDENTIALS"
)  # path to credentials file for Google Calendar API

In [ ]:
# filter widgets

header = widgets.HTML(
    value="""
        <h1 style="text-align: center; font-size: 120%">Calendar Insights</h1>
    """
)

start_date = widgets.DatePicker(
    description="Start Date", disabled=False, value=date.today() - timedelta(days=28)
)

end_date = widgets.DatePicker(
    description="End Date", disabled=False, value=date.today()
)

calendars = widgets.SelectMultiple(
    options=CALENDARS, value=CALENDARS, description="Calendars", disabled=False
)

period = widgets.Dropdown(
    options=TimePeriodType.all_to_list(), value="daily", description="Period"
)

aggregation = widgets.Dropdown(
    options=AggregationType.all_to_list(), value="total", description="Aggregation"
)

separator = widgets.HTML(value="<br>")

In [5]:
def refresh_activities():
    """Pull in calendar activities based on filter values."""
    global activities
    activities = load_calendar_data(
        calendars.value,
        ACTIVITY_REGEX,
        CREDENTIALS_FILE,
        CACHE_FILE,
        datetime.combine(start_date.value, datetime.min.time()),
        datetime.combine(end_date.value, datetime.min.time()),
    )

In [6]:
class CalendarOverTime(ActivityVisualization):
    @property
    def title(self):
        return "Time Spent in each Calendar"

    @property
    def description(self):
        return "Each calendar represents a high level category of activity."

    def process(self):
        data_by_calendar = {}
        periods = self.get_periods()

        for calendar, calendar_data in self.activities.items():
            data_by_period = {d.start: 0 for d in periods.periods}

            for activity in calendar_data:
                day_diff = activity.end.date() - activity.start.date()
                current_period = periods.get_period(activity.start.date())
                if current_period is None:
                    continue

                current_period = current_period.start
                current_date = activity.start

                for day in range(day_diff.days + 1):
                    if (
                        datetime.combine(
                            current_date + timedelta(days=1), datetime.min.time()
                        )
                        < activity.end
                    ):
                        data_by_period[current_period] += (
                            current_date + timedelta(days=1) - current_date
                        ).hours

                        next_period = periods.get_period(
                            (current_date + timedelta(days=1)).date()
                        )
                        if next_period is None:
                            continue

                        current_date = current_date + timedelta(days=1)
                        if next_period.start != current_period:
                            current_period = next_period.start
                    else:
                        data_by_period[current_period] += (
                            activity.end - current_date
                        ).seconds / 3600.0

            data_by_calendar[calendar] = list(data_by_period.values())

        self.data = data_by_calendar

In [7]:
class AreaOverTime(ActivityVisualization):
    @property
    def title(self):
        return "Time Spent in each Area"

    @property
    def description(self):
        return "Each area represents a specific project or context."

    def process(self):
        data_by_area = {}
        periods = self.get_periods()

        for calendar, calendar_data in self.activities.items():
            for activity in calendar_data:
                day_diff = activity.end.date() - activity.start.date()
                current_date = activity.start

                current_period = periods.get_period(activity.start.date())
                if current_period is None:
                    continue

                current_period = current_period.start

                if activity.name is None:
                    continue

                area = activity.name

                if area not in data_by_area:
                    data_by_area[area] = {d.start: 0 for d in periods.periods}

                for day in range(day_diff.days + 1):
                    if (
                        datetime.combine(
                            current_date + timedelta(days=1), datetime.min.time()
                        )
                        < activity.end
                    ):
                        data_by_area[area][current_period] += (
                            current_date + timedelta(days=1) - current_date
                        ).hours
                        next_period = periods.get_period(
                            (current_date + timedelta(days=1)).date()
                        )
                        if next_period is None:
                            continue

                        current_date = current_date + timedelta(days=1)
                        if next_period.start != current_period:
                            current_period = next_period.start
                    else:
                        data_by_area[area][current_period] += (
                            activity.end - current_date
                        ).seconds / 3600.0

        self.data = {category: list(d.values()) for category, d in data_by_area.items()}

In [8]:
class ContextSwitches(ActivityVisualization):
    @property
    def title(self):
        return "Total Number of Context Switches"

    @property
    def description(self):
        return "Total number of times a switch to a different activity happened."

    def process(self):
        context_switches = {d.start: 0 for d in self.get_periods().periods}

        for calendar, calendar_data in self.activities.items():
            for activity in calendar_data:
                day_diff = activity.end.date() - activity.start.date()
                current_date = activity.start

                current_period = self.get_periods().get_period(activity.start.date())
                if current_period is None:
                    continue

                current_period = current_period.start

                if activity.name is None:
                    continue

                for day in range(day_diff.days + 1):
                    if (
                        datetime.combine(
                            current_date + timedelta(days=1), datetime.min.time()
                        )
                        < activity.end
                    ):
                        context_switches[current_period] += 1
                        next_period = self.get_periods().get_period(
                            (current_date + timedelta(days=1)).date()
                        )
                        if next_period is None:
                            continue

                        current_date = current_date + timedelta(days=1)
                        if next_period.start != current_period:
                            current_period = next_period.start
                    else:
                        context_switches[current_period] += 1
        context_switches["Context Switches"] = list(context_switches.values())
        self.data = {}
        self.data["Context Switches"] = context_switches["Context Switches"]

    def aggregate_data(self):
        # just take the data as is
        pass

In [9]:
class Indicators(ActivityVisualization):
    @property
    def title(self):
        return "Quick overview of some select metrics"

    def _total_time_tracked(self, activities):
        """Total amount of time of activities that have been recorded."""
        total_time = 0
        for calendar, calendar_data in activities.items():
            for activity in calendar_data:
                total_time += activity.duration

        return total_time

    def _total_context_switches(self, activities):
        """Total numbe of context switches."""
        context_switches = 0
        for calendar, calendar_data in activities.items():
            for activity in calendar_data:
                day_diff = activity.end.date() - activity.start.date()
                current_date = activity.start

                if activity.name is None:
                    continue

                for day in range(day_diff.days + 1):
                    if (
                        datetime.combine(
                            current_date.date() + timedelta(days=1), datetime.min.time()
                        )
                        < activity.end
                    ):
                        context_switches += 1
                        current_date = current_date.date() + timedelta(days=1)
                    else:
                        context_switches += 1
        return context_switches

    def _total_areas(self, activities):
        """Total number of different areas recorded."""
        total_areas = []

        for _, calendar_data in activities.items():
            for activity in calendar_data:
                if activity.name not in total_areas:
                    total_areas.append(activity.name)

        return len(total_areas)

    def _total_activities(self, activities):
        """Total number of different activities recorded."""
        total_activities = []

        for _, calendar_data in activities.items():
            for activity in calendar_data:
                if f"{activity.name} - {activity.description}" not in total_activities:
                    total_activities.append(f"{activity.name} - {activity.description}")

        return len(total_activities)

    def process(self):
        date_diff = (self.end - self.start).days + 1
        activities_prev_period = load_calendar_data(
            CALENDARS,
            ACTIVITY_REGEX,
            CREDENTIALS_FILE,
            CACHE_FILE,
            datetime.combine(
                self.start - timedelta(days=date_diff), datetime.min.time()
            ),
            datetime.combine(self.end - timedelta(days=date_diff), datetime.min.time()),
        )

        self.data = {}

        self.data["context_switches"] = self._total_context_switches(activities)
        self.data["prev_context_switches"] = self._total_context_switches(
            activities_prev_period
        )

        self.data["time_tracked"] = self._total_time_tracked(activities)
        self.data["prev_time_tracked"] = self._total_time_tracked(
            activities_prev_period
        )

        self.data["number_of_areas"] = self._total_areas(activities)
        self.data["prev_number_of_areas"] = self._total_areas(activities_prev_period)

        self.data["number_of_activities"] = self._total_activities(activities)
        self.data["prev_number_of_activities"] = self._total_activities(
            activities_prev_period
        )

    def aggregate_data(self):
        # just take the data as is
        pass

    def plot(self):
        date_diff = (self.end - self.start).days + 1

        fig = make_subplots(
            rows=1,
            cols=4,
            specs=[
                [
                    {"type": "domain"},
                    {"type": "domain"},
                    {"type": "domain"},
                    {"type": "domain"},
                ]
            ],
        )
        fig.layout.template = "plotly_dark"

        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=self.data["context_switches"] / date_diff,
                title="Average Context Switches",
                delta={
                    "reference": self.data["prev_context_switches"] / date_diff,
                    "relative": True,
                    "position": "bottom",
                },
            ),
            row=1,
            col=1,
        )

        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=self.data["time_tracked"] / 3600,
                title="Total Time Tracked [h]",
                delta={
                    "reference": self.data["prev_time_tracked"] / 3600,
                    "relative": True,
                    "position": "bottom",
                },
            ),
            row=1,
            col=2,
        )

        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=self.data["number_of_areas"],
                title="Areas Worked On",
                delta={
                    "reference": self.data["prev_number_of_areas"],
                    "relative": True,
                    "position": "bottom",
                },
            ),
            row=1,
            col=3,
        )

        fig.add_trace(
            go.Indicator(
                mode="number+delta",
                value=self.data["number_of_activities"],
                title="Activities Worked On",
                delta={
                    "reference": self.data["prev_number_of_activities"],
                    "relative": True,
                    "position": "bottom",
                },
            ),
            row=1,
            col=4,
        )

        fig.update_layout(height=300, width=1200)
        fig.show()

In [ ]:
class TopActivities(ActivityVisualization):
    @property
    def title(self):
        return "Top Activities."

    @property
    def description(self):
        return "Top 10 activities most time was spent on."

    def process(self):
        data_by_activity = {}
        periods = self.get_periods()

        for _, calendar_data in self.activities.items():
            for activity in calendar_data:
                day_diff = activity.end.date() - activity.start.date()
                current_date = activity.start

                current_period = periods.get_period(activity.start.date())
                if current_period is None:
                    continue

                current_period = current_period.start

                if activity.description is None:
                    act = f"{activity.name}"
                else:
                    act = f"{activity.name} - {activity.description}"

                if act not in data_by_activity:
                    data_by_activity[act] = {d.start: 0 for d in periods.periods}

                for _ in range(day_diff.days + 1):
                    if (
                        datetime.combine(
                            current_date + timedelta(days=1), datetime.min.time()
                        )
                        < activity.end
                    ):
                        data_by_activity[act][current_period] += (
                            current_date + timedelta(days=1) - current_date
                        ).hours
                        next_period = periods.get_period(
                            (current_date + timedelta(days=1)).date()
                        )
                        if next_period is None:
                            continue

                        current_date = current_date + timedelta(days=1)
                        if next_period.start != current_period:
                            current_period = next_period.start
                    else:
                        data_by_activity[act][current_period] += (
                            activity.end - current_date
                        ).seconds / 3600.0

            thresholds_per_period = {p.start: [] for p in periods.periods}
            for _, values in data_by_activity.items():
                for period, d in values.items():
                    if len(thresholds_per_period[period]) < 10:
                        thresholds_per_period[period].append(d)
                    elif any(v < d for v in thresholds_per_period[period]):
                        thresholds_per_period[period].append(d)
                        thresholds_per_period[period] = [
                            v for v in thresholds_per_period[period] if v >= d
                        ]

        self.data = {
            activity: list(d.values())
            for activity, d in data_by_activity.items()
            for period, v in d.items()
            if v >= min(thresholds_per_period[period])
        }

    def aggregate_data(self):
        # just take the data as is
        pass

In [ ]:
visualizations = [
    Indicators(),
    CalendarOverTime(),
    AreaOverTime(),
    TopActivities(),
    ContextSwitches(),
]

In [10]:
def refresh(start_date, end_date, calendars, period, aggregation):
    """Refresh visualizations."""
    refresh_activities()

    # make sure status indicators are plotted first
    visualizations[0].refresh(
        activities=activities,
        start=start_date,
        end=end_date,
        calendars=calendars,
        period=period,
        aggregation=aggregation,
    )

    # plot visualizations in parallel
    for viz in visualizations:
        if not isinstance(viz, Indicators):
            Process(
                target=viz.refresh(
                    activities=activities,
                    start=start_date,
                    end=end_date,
                    calendars=calendars,
                    period=period,
                    aggregation=aggregation,
                )
            ).start()


# make filters interactive, call refresh function when changed
widget = widgets.interactive(
    refresh,
    start_date=start_date,
    end_date=end_date,
    calendars=calendars,
    period=period,
    aggregation=aggregation,
)
widget.update()
# display filters horizontally
controls = widgets.HBox(
    widget.children[:-1], layout=widgets.Layout(flex_flow="row wrap")
)
output = widget.children[-1]
display(widgets.VBox([header, controls, separator, output]))

interactive(children=(DatePicker(value=datetime.date(2022, 10, 26), description='Start Date'), DatePicker(valu…